In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Layer

In [ ]:
!git clone https://github.com/facebookresearch/detr.git
os.chdir('detr')
!git checkout a54b77800eb8e64e3ad0d8237789fcbf2f8350c5

In [ ]:
os.chdir('/kaggle/input/models-1')
from models1.face_recognition.model_small import create_model
from models1.face_recognition.align import AlignDlib
from models1.triplet_loss import TripletLossLayer


In [ ]:
os.chdir('/kaggle/input/lfw-preprocessor')
from lfw_preprocessor import LfwDataGenerator

In [ ]:
import argparse
import random
from pathlib import Path

import torch
import torchvision.transforms as T
import matplotlib.pyplot as plt
import PIL.Image

os.chdir('/kaggle/input/detr-1/detr')
import util.misc as utils
from models import build_model

In [ ]:
pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

In [ ]:
os.chdir('/kaggle/working/detr')
from main import get_args_parser

In [ ]:
os.chdir('/kaggle/input/competition')

In [ ]:
import csv

#Dictionary of IDs (int) and names (string)
idict = {}
idict2 = {}
with open("person_id_name_mapping.csv", mode = 'r') as ids:
  tdict = csv.DictReader(ids)
  for r in tdict:
    idict[r["person_name"]] =  int(r["person_id"])
    idict2[int(r["person_id"])] = r["person_name"]

print(idict)
print(idict2)


In [ ]:
in_a = Input(shape = (96, 96, 3), name = "img_a")
in_p = Input(shape = (96, 96, 3), name = "img_p")
in_n = Input(shape = (96, 96, 3), name = "img_n")

model_sm = create_model()

emb_a = model_sm(in_a)
emb_p = model_sm(in_p)
emb_n = model_sm(in_n)

triplet_loss_layer = TripletLossLayer(alpha = 0.2, name = 'triplet_loss_layer')([emb_a, emb_p, emb_n])

model = Model([in_a, in_p, in_n], triplet_loss_layer)

os.chdir('/kaggle/input/lfw-checkpoints')
model.load_weights('ckpts/epoch010_loss2.664.hdf5')

In [ ]:
base_model = model.layers[3]

In [ ]:
class IdentityMetadata():
    def __init__(self, base, name, file):
        # dataset base directory
        self.base = base
        # identity name
        self.name = name
        # image file name
        self.file = file

    def __repr__(self):
        return self.image_path()

    def image_path(self):
        return os.path.join(self.base, self.name, self.file) 
    
def load_metadata(path, upper_limit):
    metadata = []
    count = 0
    for i in sorted(os.listdir(path)):
        if count == upper_limit: 
            break
            
        for f in sorted(os.listdir(os.path.join(path, i))):
            if count == upper_limit: 
                break
                
            count += 1
            # Check file extension. Allow only jpg/jpeg' files.
            ext = os.path.splitext(f)[1]
            if ext == '.jpg' or ext == '.jpeg':
                metadata.append(IdentityMetadata(path, i, f))
    return np.array(metadata)

def load_image(path):
    img = cv2.imread(path, 1)
    # OpenCV loads images with color channels
    # in BGR order. So we need to reverse them
    return img[...,::-1]
    
def align_image(img):
        os.chdir('/kaggle/input/models-1')
        alignment = AlignDlib('models1/landmarks.dat')
        bb = alignment.getLargestFaceBoundingBox(img)
        if bb is None:
            return cv2.resize(img, (96,96))
        else:
            return alignment.align(96, 
                                   img, 
                                   bb,
                                   landmarkIndices=AlignDlib.OUTER_EYES_AND_NOSE)


In [ ]:
os.chdir('/kaggle/input/fire-pictures')
metadata = load_metadata('data/fire_data', 144)

os.chdir('/kaggle/input/models-1')
alignment = AlignDlib('models1/landmarks.dat')

In [ ]:
os.chdir('/kaggle/input/fire-pictures')

In [ ]:
embeddings = np.empty((metadata.shape[0], 128))
ids1 = []
for i, m in enumerate(metadata):
    print(m.image_path())
    img = load_image(m.image_path())
    img = align_image(img)
    print('test')
    os.chdir('/kaggle/input/fire-pictures')
    img = img.astype('float32')
    img = img / 255.0
    img = np.expand_dims(img, axis = 0)
    name = m.image_path().split('/')[2].replace('_', ' ')
    embeddings[i] = base_model.predict(img)
    if(idict.get(name)) :  
      ids1.append(idict[name])
    else:
      ids1.append(-1)
    
    if i%1 == 0: print(i, end = " ")
    print(ids1[i])
    if i == len(metadata) - 1: print('done')

In [ ]:
def distance(emb1, emb2):
    return np.sum(np.square(emb1 - emb2))

In [ ]:
def knockknock(pth):
    mini = 100
    idd = -1    
    for i in range(43):
      d = distance2(i * 3, pth)
      if d < mini and ids1[i * 3] > -1:
          idd = ids1[i * 3]
          mini = d

    return idd

In [ ]:
def distance2(img1, img2):
  return distance(embeddings[img1], predictor(img2))

In [ ]:
def predictor(pth):
  img = load_image(pth)
  img = align_image(img)
  img = img.astype('float32')
  img = img / 255.0
  img = np.expand_dims(img, axis = 0)
  return base_model.predict(img)

In [ ]:
def match(pth): 
  return idict2[knockknock(pth)]

In [ ]:
parser = argparse.ArgumentParser(description='DETR args parser', parents=[get_args_parser()])
args = parser.parse_args(args=[])
os.chdir('/kaggle/input/models-1/models1')
args.resume = 'checkpoint.pth'
args.device = 'cpu'

if args.output_dir:
  Path(args.output_dir).mkdir(parents=True, exist_ok=True)

args.distributed = False

print(args)

In [ ]:
model, criterion, postprocessors = build_model(args)

device = torch.device(args.device)
model.to(device)

In [ ]:
output_dir = Path(args.output_dir)
if args.resume:
  if args.resume.startswith('https'):
    checkpoint = torch.hub.load_state_dict_from_url(args.resume, map_location='cpu', check_hash=True)
  else:
    checkpoint = torch.load(args.resume, map_location='cpu')
  
  model.load_state_dict(checkpoint['model'], strict=True)

In [ ]:
# COCO classes
CLASSES = [
   'N/A', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
   'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A',
   'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
   'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack',
   'umbrella', 'N/A', 'N/A', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
   'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
   'skateboard', 'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass',
   'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
   'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake',
   'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table', 'N/A',
   'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
   'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A',
   'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
   'toothbrush'
]

In [ ]:
# colors for visualization
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

# standard PyTorch mean-std input image normalization
transform = T.Compose([
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# for output bounding box post-processing
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b

In [ ]:
def detect(im, model, transform):
  img = transform(im).unsqueeze(0)
  assert img.shape[-2] <= 1600 and img.shape[-1] <= 1600, 'demo model only supports images up to 1600 pixels on each side'
  outputs = model(img)
  probas = outputs['pred_logits'].softmax(-1)[0, :, :-1]
  keep = probas.max(-1).values > 0.7
  bboxes_scaled = rescale_bboxes(outputs['pred_boxes'][0, keep], im.size)
  return probas[keep], bboxes_scaled

In [ ]:
def plot_lite(pil_img, prob, reslts, classes = ['person']):
  plt.figure(figsize=(16,10))
  plt.imshow(pil_img)
  ax = plt.gca()
  for p, (text, (xmin, ymin, xmax, ymax)), c in zip(prob, reslts, COLORS * 100):
    cl = p.argmax()
    if CLASSES[cl] not in classes:
      continue
    ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, fill=False, color=c, linewidth=3))
    
    ax.text(xmin, ymin, text, fontsize=15, bbox=dict(facecolor='yellow', alpha=0.5))
  plt.axis('off')
  plt.show
  plt.close

In [ ]:
def detect_and_draw(ipth):
  dimg = PIL.Image.open(ipth)
  prob, boxs = detect(dimg, model, transform)
  outs = []

  if not os.path.isdir("/kaggle/working"):
    os.mkdir("/kaggle/working") 
    
  for i,j in enumerate(boxs.tolist()):
    copped = dimg.crop(j)
    cpth = "/kaggle/working/IMG_" + str(i) + ".jpg"
    copped.save(cpth)
    outs.append([cpth, j])

  # Replace f with the function that takes in a file path and returns the name of the person in the image
  # like the filepath will be the cropped face (by itself) saved as a .jpg, so the function just has to tell who it is

  # i.e. if the function is called "make_name" then replace
  #      the current definition of f with
  #      f = make_name

  # Currently it does nothing

  f = match
  outs = [(f(path), dims) for path,dims in outs]

  return dimg, prob, outs
  #return outs

In [ ]:
os.chdir('/kaggle/input/competition')

In [ ]:
img, prob, result_a = detect_and_draw('a.jpg')
plot_lite(img, prob, result_a)

In [ ]:
os.chdir('/kaggle/input/competition')

In [ ]:
img, prob, result_b = detect_and_draw('b.jpg')
plot_lite(img, prob, result_b)

In [ ]:
os.chdir('/kaggle/input/competition')

In [ ]:
img, prob, result_c = detect_and_draw('c.jpg')
plot_lite(img, prob, result_c)

In [ ]:
os.chdir('/kaggle/input/competition')

In [ ]:
img, prob, result_d = detect_and_draw('d.jpg')
plot_lite(img, prob, result_d)

In [ ]:
def csv_rite(fles):
  heads = ["id", "xmin", "xmax", "ymin", "ymax"]

  with open('/kaggle/working/copped.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(heads)

    for rslt in fles:
      for ln in rslt[1]:
        k = [rslt[0] + "_" + str(idict[ln[0]])]
        k.extend(list(map(int, ln[1])))
        x = k[2]
        k[2] = k[3]
        k[3] = x
        print(k)
        writer.writerow(k)

In [ ]:
csv_rite([["a", result_a]])

In [ ]:
csv_rite([["a", result_a], ["b", result_b], ["c", result_c], ["d", result_d]])

In [ ]:
import csv

In [ ]:
def csv_rite2(ourdata, sampledata):
  odict = {}
  with open(sampledata, mode = 'r') as smpl:
    tdict = csv.DictReader(smpl)
    for r in tdict:
      odict[r["id"]] =  [0, 0, 0, 0]

  with open(ourdata, mode = 'r') as smpl:
    tdict = csv.DictReader(smpl)
    for r in tdict:
      if odict.get(r["id"]):
        odict[r["id"]] =  [r["xmin"], r["xmax"], r["ymin"], r["ymax"]]

  heads = ["id", "xmin", "xmax", "ymin", "ymax"]

  print(odict)

  with open('/kaggle/working/output.csv', mode='w', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(heads)

    for (pids, crds) in odict.items():
      k = [pids]
      k.extend(crds)
      #print(k)
      writer.writerow(k)
 

In [ ]:
os.chdir('/kaggle')

In [ ]:
os.getcwd()

In [ ]:
!ls -a

In [ ]:
csv_rite2("working/copped.csv", "input/sample-submission/kaggle_sample_submission.csv")